In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [50]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'
my_cache = cache_state('backtest_uni')

def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


INFO:	Cache Pointing to: backtest_uni


True
[]
[<Process(Process-14, started)>]


INFO:	Cache Pointing to: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	kite_simulator: 1
DEBUG:	ohlc_tick_handler: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["TCS", "WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T", "mode": "full"}
DEBUG:	Trade_init: TCS
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
DEBUG:	Trade_init: WIPRO
INFO:	WIPRObacktest_uni=>{'stock': 'WIPRO', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L

TRADE: B: EN:  : 2019-04-08 13:50 : TCS -> 2054.75
TRADE: S: EN:  : 2019-04-08 13:52 : WIPRO -> 261.55
TRADE: B: EX:  : 2019-04-08 13:53 : WIPRO -> 261.65
TRADE: S: EX:  : 2019-04-08 13:55 : TCS -> 2057.75
TRADE: S: EN:  : 2019-04-08 13:55 : WIPRO -> 261.7
TRADE: B: EN:  : 2019-04-08 13:57 : TCS -> 2058.0
TRADE: S: EX:  : 2019-04-08 13:58 : TCS -> 2057.7
TRADE: B: EN:  : 2019-04-08 14:00 : TCS -> 2058.75
TRADE: B: EX:  : 2019-04-08 14:01 : WIPRO -> 261.5
TRADE: S: EX:  : 2019-04-08 14:04 : TCS -> 2066.0
TRADE: S: EN:  : 2019-04-08 14:07 : TCS -> 2066.85
TRADE: S: EN:  : 2019-04-08 14:08 : WIPRO -> 262.35
TRADE: B: EX:  : 2019-04-08 14:09 : WIPRO -> 262.4
TRADE: S: EN:  : 2019-04-08 14:11 : WIPRO -> 262.3
TRADE: B: EX:  : 2019-04-08 14:15 : WIPRO -> 262.25
TRADE: B: EX:  : 2019-04-08 14:16 : TCS -> 2059.0
TRADE: S: EN:  : 2019-04-08 14:19 : TCS -> 2060.35
TRADE: S: EN:  : 2019-04-08 14:19 : WIPRO -> 262.4
TRADE: B: EX:  : 2019-04-08 14:22 : TCS -> 2059.95
TRADE: B: EX:  : 2019-04-08 14:

In [51]:
backtest_unitest(['TCS','WIPRO'])

In [49]:
################################
###         Stop Trade       ###
################################

my_cache.delete('msgBufferQueue'+cache_postfix)
#my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())

jobs
jobs = []

False
[]


In [43]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [53]:
cache_state('backtest_uni').getValue()

INFO:	Cache Pointing to: backtest_uni


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,1,1,20661.0,0,60.49999999999727,2.936821921797882,153.999999999985,0.7453656647789797,...,2073.6,0.0,0,2075.0,2019-04-08 15:29,SCANNING,PAUSE,haikin_1_new,1T,minute
1,WIPRO,10,1,1,2635.5,0,-6.999999999999886,-2.6631158455392376,-2.5000000000022737,-0.09485866059579867,...,263.6,0.0,0,264.6,2019-04-08 15:29,SCANNING,PAUSE,haikin_1_new,1T,minute


In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

In [ ]:
buy = pd.DataFrame()

if buy.shape[0] > 0 and buy[-1]==True:
    print('Yeah')

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
my_cache = cache_state('backtest_uni')

In [ ]:
df = my_cache.getValue()

In [ ]:
pd.set_option('display.max_rows', 1000)
my_cache.getOHLC('TCS')

In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

In [ ]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

In [5]:
from lib.data_model_lib import *
live_cache = cache_state('live')

INFO:	Cache Pointing to: live


In [3]:
from kiteconnect import KiteConnect

In [4]:
kite_api_key = 'b2w0sfnr1zr92nxm'

live_cache.set('KiteAPIKey',kite_api_key)

True

In [5]:
live_cache.get('access_token')

'o2MI2Q7xSNPVSZOy24mQx5h28BL765Mv'

In [6]:
live_cache.get('KiteAPIKey')

'b2w0sfnr1zr92nxm'

In [7]:
KiteAPIKey = live_cache.get('KiteAPIKey')
kite = KiteConnect(api_key=KiteAPIKey)
access_token = live_cache.get('access_token')
kite.set_access_token(access_token)

In [8]:
kite.profile()

{'user_id': 'YM1755',
 'user_type': 'individual',
 'email': 'saswati.suhan@gmail.com',
 'user_name': 'Saswati Saha',
 'user_shortname': 'Saswati',
 'broker': 'ZERODHA',
 'exchanges': ['BSE', 'MCX', 'NSE', 'NFO', 'MF', 'CDS', 'BFO'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': '',
 'meta': {'demat_consent': 'physical'}}

In [9]:
instruments_df = pd.DataFrame(kite.instruments())

In [67]:
kite.orders()

[{'placed_by': 'YM1755',
  'order_id': '200511002296687',
  'exchange_order_id': '1000000008525391',
  'parent_order_id': None,
  'status': 'COMPLETE',
  'status_message': None,
  'status_message_raw': None,
  'order_timestamp': datetime.datetime(2020, 5, 11, 12, 31, 16),
  'exchange_update_timestamp': '2020-05-11 12:31:16',
  'exchange_timestamp': datetime.datetime(2020, 5, 11, 12, 31, 16),
  'variety': 'regular',
  'exchange': 'NSE',
  'tradingsymbol': 'CIPLA',
  'instrument_token': 177665,
  'order_type': 'LIMIT',
  'transaction_type': 'SELL',
  'validity': 'DAY',
  'product': 'MIS',
  'quantity': 1,
  'disclosed_quantity': 0,
  'price': 586.55,
  'trigger_price': 0,
  'average_price': 586.55,
  'filled_quantity': 1,
  'pending_quantity': 0,
  'cancelled_quantity': 0,
  'market_protection': 0,
  'meta': {},
  'tag': None,
  'guid': '01XV6OchBiiVM2W'},
 {'placed_by': 'YM1755',
  'order_id': '200511002303419',
  'exchange_order_id': '1000000008557450',
  'parent_order_id': None,
  'st

In [59]:
kite.positions()

{'net': [{'tradingsymbol': 'CIPLA',
   'exchange': 'NSE',
   'instrument_token': 177665,
   'product': 'MIS',
   'quantity': 0,
   'overnight_quantity': 0,
   'multiplier': 1,
   'average_price': 0,
   'close_price': 0,
   'last_price': 585.6,
   'value': 0.7999999999997272,
   'pnl': 0.7999999999997272,
   'm2m': 0.7999999999997272,
   'unrealised': 0.7999999999997272,
   'realised': 0,
   'buy_quantity': 3,
   'buy_price': 586.1333333333333,
   'buy_value': 1758.4,
   'buy_m2m': 1758.4,
   'sell_quantity': 3,
   'sell_price': 586.4,
   'sell_value': 1759.1999999999998,
   'sell_m2m': 1759.1999999999998,
   'day_buy_quantity': 3,
   'day_buy_price': 586.1333333333333,
   'day_buy_value': 1758.4,
   'day_sell_quantity': 3,
   'day_sell_price': 586.4,
   'day_sell_value': 1759.1999999999998},
  {'tradingsymbol': 'SUNPHARMA',
   'exchange': 'NSE',
   'instrument_token': 857857,
   'product': 'MIS',
   'quantity': 0,
   'overnight_quantity': 0,
   'multiplier': 1,
   'average_price': 0,
 

In [40]:
kite.trades()

[{'trade_id': '2444958',
  'order_id': '200511002296687',
  'exchange': 'NSE',
  'tradingsymbol': 'CIPLA',
  'instrument_token': 177665,
  'product': 'MIS',
  'average_price': 586.55,
  'quantity': 1,
  'exchange_order_id': '1000000008525391',
  'transaction_type': 'SELL',
  'fill_timestamp': datetime.datetime(2020, 5, 11, 12, 31, 16),
  'order_timestamp': '12:31:16',
  'exchange_timestamp': datetime.datetime(2020, 5, 11, 12, 31, 16)},
 {'trade_id': '2455345',
  'order_id': '200511002303419',
  'exchange': 'NSE',
  'tradingsymbol': 'CIPLA',
  'instrument_token': 177665,
  'product': 'MIS',
  'average_price': 586.25,
  'quantity': 1,
  'exchange_order_id': '1000000008557450',
  'transaction_type': 'BUY',
  'fill_timestamp': datetime.datetime(2020, 5, 11, 12, 32, 11),
  'order_timestamp': '12:32:11',
  'exchange_timestamp': datetime.datetime(2020, 5, 11, 12, 32, 11)},
 {'trade_id': '2477143',
  'order_id': '200511002313172',
  'exchange': 'NSE',
  'tradingsymbol': 'CIPLA',
  'instrument_

In [12]:
instruments_df.to_hdf('data/instruments.hdf',key='instruments')

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['exchange_token', 'tradingsymbol', 'name', 'expiry', 'instrument_type',
       'segment', 'exchange'],
      dtype='object')]

  encoding=encoding,


In [14]:
instruments_df.columns

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')

In [21]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [25]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

array([2953217,  969473])

In [35]:
kite.margins()

{'equity': {'enabled': True,
  'net': 2182.61,
  'available': {'adhoc_margin': 0,
   'cash': 2182.61,
   'opening_balance': 2182.61,
   'live_balance': 2182.61,
   'collateral': 0,
   'intraday_payin': 0},
  'utilised': {'debits': 0,
   'exposure': 0,
   'm2m_realised': 0,
   'm2m_unrealised': 0,
   'option_premium': 0,
   'payout': 0,
   'span': 0,
   'holding_sales': 0,
   'turnover': 0,
   'liquid_collateral': 0,
   'stock_collateral': 0}},
 'commodity': {'enabled': True,
  'net': 0,
  'available': {'adhoc_margin': 0,
   'cash': 0,
   'opening_balance': 0,
   'live_balance': 0,
   'collateral': 0,
   'intraday_payin': 0},
  'utilised': {'debits': 0,
   'exposure': 0,
   'm2m_realised': 0,
   'm2m_unrealised': 0,
   'option_premium': 0,
   'payout': 0,
   'span': 0,
   'holding_sales': 0,
   'turnover': 0,
   'liquid_collateral': 0,
   'stock_collateral': 0}}}

In [95]:
live_cache.publish('live_trade_handlerlive','INIT')

1

In [34]:
live_cache.publish('live_trade_handlerlive','CLOSE')

1

In [36]:
live_cache.publish('live_trade_handlerlive','START')

1

In [96]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'add','value':[969473, 2953217],'mode':'quote'}))

1

In [10]:
import json

In [35]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'remove','value':[969473, 2953217],'mode':'quote'}))

3

In [74]:
live_cache.publish('live_trade_handlerlive','add')

1

In [31]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':187.5,'qty':1}))

1

In [32]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':184.6,'qty':1}))

1

In [21]:
json.dumps({'cmd':'buy','symbol':'WIPRO','price':187.5,'qty':1})

'{"cmd": "buy", "symbol": "WIPRO", "price": 187.5, "qty": 1}'

In [66]:
buy_limit('WIPRO',186.5,1)

In [62]:
sell_limit('WIPRO',187.2,1)

In [65]:
def buy_limit(symbol, price, quantity=1): 
    pinfo("B Limit: {}[{}]=> {}".format(symbol, quantity, price))

    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_LIMIT,
                                product=kite.PRODUCT_MIS,
                                #trigger_price=round(trigger,1),
                                #stoploss=round(stoploss,1),
                                #trigger_price=round(price,1),
                                price=price,
                                variety=kite.VARIETY_REGULAR,
                                tag='freedom_v2'
                                )
        pinfo("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        pinfo("Order placement failed: {}".format(e.message))
        
def sell_limit(symbol, price, quantity=1):
    pinfo("S Limit: {}[{}]=> {}".format(symbol, quantity, price))

    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_LIMIT,
                            product=kite.PRODUCT_MIS,
                            #trigger_price=round(trigger,1),
                            #trigger_price=round(price,1),
                            price=price,
                            variety=kite.VARIETY_REGULAR)
        pinfo("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        pinfo("Order placement failed: {}".format(e.message))

In [61]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

'200511002924441'

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [69]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


'200511003019322'

In [11]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [12]:
instruments_df

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,540584966,2111660,EURINR20DEC77.7500CE,EURINR,0.0,2020-12-29,77.75,0.0025,1,CE,BCD-OPT,BCD
1,540578822,2111636,EURINR20DEC77.7500PE,EURINR,0.0,2020-12-29,77.75,0.0025,1,PE,BCD-OPT,BCD
2,540512774,2111378,EURINR20DEC78.0000CE,EURINR,0.0,2020-12-29,78.00,0.0025,1,CE,BCD-OPT,BCD
3,540506374,2111353,EURINR20DEC78.0000PE,EURINR,0.0,2020-12-29,78.00,0.0025,1,PE,BCD-OPT,BCD
4,540627462,2111826,EURINR20DEC78.2500CE,EURINR,0.0,2020-12-29,78.25,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
64873,978945,3824,ZODJRDMKJ,ZODIAC JRD-MKJ,0.0,,0.00,0.0500,1,EQ,NSE,NSE
64874,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.00,0.0500,1,EQ,NSE,NSE
64875,7437825,29054,ZUARI-BE,ZUARI AGRO CHEMICALS,0.0,,0.00,0.0500,1,EQ,NSE,NSE
64876,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,,0.00,0.0500,1,EQ,NSE,NSE


In [13]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [14]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

,tradingsymbol
instrument_token,
1749249,1018GS2026-GS
1745665,1025GS2021-GS
1744385,116GS2020-GS
4331777,20MICRONS
1025,21STCENMGM
...,...
978945,ZODJRDMKJ
2916865,ZOTA
7437825,ZUARI-BE


In [15]:
live_cache.delete('eq_token')

0

In [64]:
eq_nse.head().to_dict(orient='records')

[{'instrument_token': 1749249, 'tradingsymbol': '1018GS2026-GS'},
 {'instrument_token': 1745665, 'tradingsymbol': '1025GS2021-GS'},
 {'instrument_token': 1744385, 'tradingsymbol': '116GS2020-GS'},
 {'instrument_token': 4331777, 'tradingsymbol': '20MICRONS'},
 {'instrument_token': 1025, 'tradingsymbol': '21STCENMGM'}]

In [70]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

[{'1018GS2026-GS': 1749249,
  '1025GS2021-GS': 1745665,
  '116GS2020-GS': 1744385,
  '20MICRONS': 4331777,
  '21STCENMGM': 1025}]

In [76]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

{1749249: '1018GS2026-GS',
 1745665: '1025GS2021-GS',
 1744385: '116GS2020-GS',
 4331777: '20MICRONS',
 1025: '21STCENMGM'}

In [16]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

True

In [17]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

['2953217', '969473']

In [10]:
live_cache.hgetall('eq_token')

{}

In [37]:
live_cache.hget('eq_token',[2953217, 969473])

DataError: Invalid input of type: 'list'. Convert to a bytes, string, int or float first.

In [42]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [43]:
msg['data']

'[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'

In [45]:
val = json.loads(msg['data'])

In [58]:
for data in val:
    instrument_token = data['instrument_token']
    stock_id = live_cache.hmget('eq_token',instrument_token)[0]


    ltp = data['last_price']
    #temp_df = pd.DataFrame(data={'date':[date_val],'ltp':[ltp]})
    #temp_df = temp_df.set_index('date')
    #temp_df.index = pd.to_datetime(temp_df.index)
    print(stock_id)
    print(ltp)
    hash_key = stock_id

WIPRO
184.7
TCS
1935.3


In [55]:
stock_id

['WIPRO']

In [94]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,WIPRO,1,1,1,0,0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1999-01-01,INIT,PAUSE,haikin_1_new,1T,minute
1,TCS,1,1,1,0,0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1999-01-01,INIT,PAUSE,haikin_1_new,1T,minute


In [93]:
live_cache.add('WIPRO')

In [90]:
stock_key = 'TCS'
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')

0

In [86]:
live_cache.add('TCS')

In [82]:
live_cache.remove()

In [6]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,WIPRO,1,1,1,0,0,0,0.0,0,0.0,...,184.7,0.0,0.0,184.7,2020-05-11 10:26,SCANNING,PAUSE,haikin_1_new,1T,minute
1,TCS,1,1,1,0,0,0,0.0,0,0.0,...,1935.3,0.0,0.0,1935.3,2020-05-11 10:26,SCANNING,PAUSE,haikin_1_new,1T,minute


In [18]:
live_cache.hmget('eq_token','TCS')[0]

'2953217'